## 프로젝트 - Movielens 영화 추천 실습
MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

>1) wget으로 데이터 다운로드 >> $ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

>2) 다운받은 데이터를 작업디렉토리로 이동
>$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

>3) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. 
>$ unzip ml-1m.zip

---
## 1. 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

In [8]:
import os
import pandas as pd

rating_file_path = os.getenv('HOME') + '/Development/ML/Modu_Lab/recommendata_iu/ml_data/ratings.dat'
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=rating_cols, engine='python')
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [37]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [38]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path = os.getenv('HOME') + '/Development/ML/Modu_Lab/recommendata_iu/ml_data/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. EDA
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [39]:
print(ratings.shape, movies.shape)    # 각 데이터 shape 확인
ratings['user_id'].nunique()     # unique한 id의 개수

(836478, 4) (3883, 3)


6039

In [42]:
# ratings와 movies를 합친다.
df = pd.merge(ratings, movies, on='movie_id')   # 'movie_id'를 기준으로 merge on 한다.
print(df.shape)
df.head()

(836478, 6)


,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [99]:
# 결측치 확인
# df.info()
df.isnull().sum()

user_id      0
movie_id     0
count        0
timestamp    0
title        0
genre        0
dtype: int64

In [102]:
# 인기 많은 영화 (많은 유저가 본 영화들)
# movie_count = df['user_id'].groupby(df['title']).count()
movie_count = df.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(10)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Name: user_id, dtype: int64

In [92]:
# 유저별 관람 영화 통계 ???
user_count = df.groupby('user_id')['count'].median()
user_count.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

In [115]:
# 영화별 평균 평점
df.groupby('title').mean().sort_values('count', ascending=False).head(20)

,user_id,movie_id,count,timestamp
title,,,,
Ulysses (Ulisse) (1954),3485.000000,3172.0,5.0,9.670604e+08
Country Life (1994),1631.000000,687.0,5.0,9.747256e+08
Schlafes Bruder (Brother of Sleep) (1995),1915.000000,989.0,5.0,9.746939e+08
Foreign Student (1994),3618.000000,572.0,5.0,9.671202e+08
Follow the Bitch (1998),2869.000000,1830.0,5.0,9.724390e+08
One Little Indian (1973),5851.000000,3607.0,5.0,9.577566e+08
Criminal Lovers (Les Amants Criminels) (1999),850.000000,3800.0,5.0,9.753594e+08
Message to Love: The Isle of Wight Festival (1996),3951.200000,1420.0,5.0,9.666918e+08
Identification of a Woman (Identificazione di una donna) (1982),5948.000000,1360.0,5.0,1.016564e+09


In [155]:
# user수가 1인 영화 리스트
condition = movie_count.sort_values() < 2
# movie_count.sort_values(ascending=False, ignore_index=condition.values).head(10)

condition1 = df.groupby('title')['user_id'].count() < 2
condition1

title
$1,000,000 Duck (1971)                        False
'Night Mother (1986)                          False
'Til There Was You (1997)                     False
'burbs, The (1989)                            False
...And Justice for All (1979)                 False
                                              ...  
Zed & Two Noughts, A (1985)                   False
Zero Effect (1998)                            False
Zero Kelvin (Kj�rlighetens kj�tere) (1995)    False
Zeus and Roxanne (1997)                       False
eXistenZ (1999)                               False
Name: user_id, Length: 3628, dtype: bool

In [116]:
import pandas as pd
import numpy as np

test = pd.DataFrame({
    'city': ['부산', '부산', '부산', '부산', '서울', '서울', '서울'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})
test

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4
4,서울,apple,150,5
5,서울,apple,200,6
6,서울,banana,400,7


In [128]:
test.groupby('city').mean()

,price,quantity
city,,
부산,212.5,2.5
서울,250.0,6.0


In [119]:
# 도시(city)와 과일(fruits)로 평균
test.groupby(['city', 'fruits']).mean()

price  quantity
city fruits                 
부산   apple   100.0       1.0
     banana  275.0       3.5
     orange  200.0       2.0
서울   apple   175.0       5.5
     banana  400.0       7.0

In [125]:
# city와 fruits 별로 평균을 정리하고 price평균 기준으로 정렬후 인덱스를 reset
test.groupby(['city', 'fruits'], as_index=False).mean().sort_values('price').reset_index(drop=True)


,city,fruits,price,quantity
0,부산,apple,100.0,1.0
1,서울,apple,175.0,5.5
2,부산,orange,200.0,2.0
3,부산,banana,275.0,3.5
4,서울,banana,400.0,7.0


In [126]:
# 그룹 안에 데이터를 확인하고 싶은 경우 사용합니다.
test.groupby('city').get_group('부산')

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4
